In [307]:
import torch
import random
import numpy as np


import pycolmap

from dataclasses import dataclass

from double_sol import Camera
from utils.rotation_utils import get_random_upward, get_upward_with_dev, get_rt_mtx
from scipy.spatial.transform import Rotation

np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [2]:
@dataclass
class Config:
    img_width: int = 640
    img_height: int = 640
    focal_length: int = (img_width * 0.5) / np.tan(60.0 * np.pi / 180.0);
    min_depth: float = 1.
    max_depth: float = 10.
    inliers_ratio: float = 1.
    outlier_dist: float = 30.
    
    # [TODO][IMPORTNAT]: not properly tested, be aware of using for
    # some experiments
    pixel_noise: float = 0.

conf = Config()

def get_random_image_point(conf: Config):
    x = random.uniform(0, conf.img_width)
    y = random.uniform(0, conf.img_height)
    x = torch.tensor([x, y], dtype=torch.float64)    
    return x

def to_homogeneous(x):
    return torch.cat([x, torch.ones(1)])

def to_camera_coords(x: torch.tensor, conf: Config = conf):
    x = to_homogeneous(x)
    
    x[0] -= conf.img_width // 2
    x[1] -= conf.img_height // 2
    x[:2] /= conf.focal_length
    x /= x.norm()
    
    return x

def generate_correspondence(x: torch.tensor, conf: Config):
    x = to_camera_coords(x, conf)
    x *= random.uniform(conf.min_depth, conf.max_depth)
    
    assert x.shape == (3,)    
    return x

def transform_correspondence(X: torch.tensor, R: torch.tensor, t: torch.tensor):
    return R.T @ (X - t)

def generate_example(R, t, conf: Config = conf):
    x1, x2 = get_random_image_point(conf), get_random_image_point(conf)
    X1, X2 = generate_correspondence(x1.clone(), conf),\
             generate_correspondence(x2.clone(), conf)
    X1, X2 = transform_correspondence(X1, R, t), transform_correspondence(X2, R, t)
    
    # [TODO][IMPORTNAT]: not properly tested, be aware of using for
    # some experiments
    if conf.pixel_noise != 0:
        x1noise = np.random.normal(0, conf.pixel_noise, 2)
        x2noise = np.random.normal(0, conf.pixel_noise, 2)
        
        if torch.all(x1[0] + x1noise > 0) and torch.all(x1[1] + x1noise < conf.img_width):
            x1 += x1noise
            assert x1[0] > 0 and x1[1] < conf.img_width, f"{x1}"
            
        if torch.all(x2[0] + x2noise > 0) and torch.all(x2[1] + x2noise < conf.img_height):
            x2 += x2noise
            assert x2[0] > 0 and x2[1] < conf.img_height, f"{x2}"
        
        assert x1[0] > 0 and x1[1] < conf.img_width, f"{x1}"
        assert x2[0] > 0 and x2[1] < conf.img_height, f"{x2}"

    return x1, x2, X1, X2 



In [296]:
from typing import Tuple
def generate_examples(num_of_examples: int,
                      dev: Tuple[float, float] = (0., 0.),
                      sim_idx = None, 
                      conf: Config = conf
                     ):
    num_of_examples = num_of_examples // 2
    
    num_inliers = num_of_examples * conf.inliers_ratio
    num_outliers = num_of_examples - num_inliers
    
    if num_of_examples == 0:
        num_of_examples, num_inliers, num_outliers = 1, 1, 0
    
    if sim_idx is not None:
        R, rand_angle = get_upward_with_dev(sim_idx, *dev), sim_idx
    else:
        R, rand_angle = get_random_upward(*dev)
    t = torch.rand(3, )

    # [TODO] [IMPORTANT]: under such generation we cannot get model where one of the points is an inlier
    xs, Xs, inliers = [], [], []
    for i in range(num_of_examples):
        x1, x2, X1, X2 = generate_example(R, t)
        Xs.append((X1, X2))

        if i < num_inliers:
            xs.append((x1, x2))
            inliers.append(True)
        else:
            xs.append((generate_outlier(x1, conf), generate_outlier(x2, conf)))
            inliers.append(False)
            
    xs = np.concatenate([[p.numpy() for p in elm] for elm in xs])
    Xs = np.concatenate([[p.numpy() for p in elm] for elm in Xs])
    
    return xs, Xs, inliers, R, t.numpy()

def colmap_to_scipy(q):
    return [*q[1:], q[0]]

def scipy_to_colmap(q):
    return [q[-1], *q[:-1]]

In [297]:
pts2d, pts3d, inlier_mask, R, t = generate_examples(10)

In [312]:
print(Rotation.from_matrix(R).as_euler("XYZ", degrees=True))

[83.  0.  0.]


In [316]:
camera = pycolmap.Camera(
    model='SIMPLE_PINHOLE',
    width=conf.img_width,
    height=conf.img_height,
    params=[conf.focal_length, conf.img_width // 2, conf.img_height // 2],
)

In [329]:
for dev_x in range(-20, 20, 1):
    for dev_z in range(-20, 20, 1):
        # print(Rotation.from_matrix(get_rt_mtx(dev_x, 0, dev_z, "cpu", torch.float64)).as_euler("XYZ", degrees=True))
        Rnoise = get_rt_mtx(dev_x, 0, dev_z, "cpu", torch.float64)
        Rnew = R @ Rnoise
        
        answer = pycolmap.pose_refinement(
            Rnoise @ t, scipy_to_colmap(Rotation.from_matrix(Rnew).as_quat()),
            pts2d, pts3d,
            [True for _ in range(pts2d.shape[0])],
            camera
        )
        
        q, t = answer['qvec'], answer['tvec']
        if not np.allclose(
            Rotation.from_quat(colmap_to_scipy(q)).as_euler("XYZ", degrees=True),
            Rotation.from_matrix(R).as_euler("XYZ", degrees=True),
            atol=1
        ):
            print("xdev: ", dev_x, " zdev: ", dev_z)
            print(Rotation.from_quat(colmap_to_scipy(q)).as_euler("XYZ", degrees=True))

xdev:  12  zdev:  -20
[74.782  4.214  8.449]
xdev:  13  zdev:  -20
[74.787  4.209  8.452]
xdev:  14  zdev:  -20
[74.771  4.227  8.441]
xdev:  14  zdev:  -18
[74.784  4.212  8.45 ]
xdev:  15  zdev:  -20
[74.771  4.226  8.441]
xdev:  15  zdev:  -18
[74.781  4.216  8.448]
xdev:  16  zdev:  -19
[74.817  4.177  8.469]
xdev:  16  zdev:  -17
[74.783  4.213  8.45 ]
xdev:  17  zdev:  -18
[74.813  4.18   8.468]
xdev:  17  zdev:  -16
[74.79   4.205  8.455]
xdev:  18  zdev:  -17
[74.77   4.228  8.44 ]
xdev:  18  zdev:  -15
[74.811  4.182  8.467]
xdev:  19  zdev:  -17
[74.781  4.218  8.439]
xdev:  19  zdev:  -15
[74.784  4.213  8.45 ]


In [330]:
pycolmap.absolute_pose_estimation(
    pts2d, pts3d, camera,
    estimation_options={'ransac': {'max_error': 12.0}},
    refinement_options={'refine_focal_length': True},
)

{'success': True,
 'qvec': array([ 0.749,  0.663, -0.   ,  0.   ]),
 'tvec': array([0.516, 0.695, 0.298]),
 'num_inliers': 10,
 'inliers': [True, True, True, True, True, True, True, True, True, True]}